In [1]:
import numpy as np 
import random 
from network import Network
from sklearn.utils.extmath import softmax

def _create_population(nn_sturcture, size=100):
    """
    size: population size, number of chromesomes 

    return population, which is a list of random created network graphs
    """
    population = []
    for _ in range(size):
        nn = Network(nn_sturcture)
        population.append(nn)
        print("population id", _)
    return population

def encode(weights, bias):
    """
    encode genes to a chromosome
    """
    gene=[]
    assert len(weights)==len(bias), "length of weiths not equal to length of bias"
    n_layers = len(bias)
    for i in range(n_layers):
        gene.extend(weights[k].flatten())
        gene.append(bias[i])
    return gene 
         
def decode(gene,structure):
    """
    decoed genes of a chromosome
    structure is the network structure, e.g.[num_inputs, 10,20,30]
    """
    Weights ={}
    Bias={}
    
    gene_index = 0
    for index in range(len(structure)):
        weight=np.zeros((structure[index],structure[index+1]))
        bias =[]
        for i in range(structure[index]):
            for j in range(structure[index+1]):
                weight[i][j] = gene[gene_index]
                gene_index+=1
        bias.append(gene[gene_index])
        gene_index+=1
        Weights[index] = weight
        Bias[index] = bias
        
    return Weights, Bias
           
    
def mutate(chro, mutation_rate):
    #for each gene, it can decide to change itself to the mean of its nearby genes or not
    new_chro = []
        
    for i in range(len(chro)-1):
        xx = (chro[i]+chro[i-1]+chro[i+1] )/3
        new_chro.append(xx)
    return new_chro
    

def crossover(chro1, chro2):
    # create a random crossover point
    assert len(chro1) ==len(chro2), "chromosome not the same length"
    index = np.random.randint(low=1, high=len(chro1))
    temp1 = chro1[index:]
    temp3 = chro1[:index]
    temp2 = chro2[:index]
    temp4 = chro2[index:]
    #random combine half of chro1 genes and half of chro2 genes below
    rd = np.random.choice([1,2,3,4])
    if rd ==1:
        new_chro = temp1.extend(temp2)
    if rd ==2:
        new_chro = temp1.extend(temp4)
    if rd ==3:
        new_chro = temp3.extend(temp2)
    if rd==4:
        new_chro = temp3.extend(temp4)
    return new_chro
    

def selection(parents,pop):
    chirldren=[] #get two chirldren
    chrome={}
    for i in range(len(parents)):
        parent_id = parents[i][0]
        nn=pop[parent_id]
        weight = nn.getWeights()
        bias = nn.getBias()
        gene = encode(weights, bias)
        chrome[i] = gene
    
    chirldren[0] = crossover(chrome[0], chrome[1])
    chirldren[1] = crossover(chrome[2],chrome[3])
    
    return chirldren
           
         
def GeneticAlgorithmTrainner(X,y, nn_structure, episodes = 10000, mutation_rate = 0.1):
    # first generation 
    pop = _create_population(nn_structure)
    fit_score= []
    
    avg_fit_scores=[]
    for i in range(len(pop)):
        nn = pop[i]
        score = fitness(nn,X,y)
        fit_score.append([i,score])
    
    fit_score=np.array(fit_score)
    fit_score[np.argsort(fit_score[:, 1])]
    fit_score=np.flip(fit_score,0)
    
    len_pop=[]
    
    # select the nn with hightest 4 fitness scores
    for epi in range(episodes):
        
            
        sc = mean(fit_score[:,1])
        avg_fit_scores.append(sc)
        print("fitness score",  sc)
        
        luckyest = fit_score[:4]
        
        #remove two worst case
#         worst2= fit_score[-2:]
#         for s in worst2:
#             del pop[s[0]]
        offspring = selection(luckyest, pop)
        
        for kid in offspring:
            #mutation
            p = np.random.random_sample()
            if p <mutation_rate:
                kid = mutate(kid, mutation_rate )
            Weights, Bias = decode(kid)
            nn = Network(nn_structure[0])
            nn.setWeights(Weights)
            nn.setBias(Bias)
            pop.append(nn)
        fit_score= []
        
        
        len_pop.append(len(pop))
        if len_pop[epi] ==len_pop[epi-1]:
            #converged 
            print("converged")
            break 
            
            
            
        #calculate new fitness score 
        for i in range(len(pop)):
            nn = pop[i]
            score = fitness(nn,X,y)
            fit_score.append([i,score])
        fit_score=np.array(fit_score)
        fit_score[np.argsort(fit_score[:, 1])]
        fit_score=np.flip(fit_score,0)

    return avg_fit_scores
        
def fitness(nn, X, y):
    ## 1/ cross_entropy 
    W = nn.getWeights()  #W is a dict, W[i] = [[n_layer_nodes, n-1_layer_nodes]]
    b = nn.getBias() # b is a dict, b[i] = [bias of layer n]
    z=[]
    a = []
    probs = []
    n_layer = len(W)


    
    for i in range(n_layer):
        print("x shape", X.shape, "y shape", y.shape)
        print("w shape", W[i].shape, "bshape", b[i].shape)
        
        if i ==0: 
            z = np.matmul(X, W[i])+b[i]
            a = np.tanh(z)
        elif i< n_layer-1:
            z = np.matmul(a,W[i])+b[i]
            a = np.tanh(z)
        else: # softmax
            z = np.matmul(a, W[i])+b[i]
            probs = softmax(z)
    probs = np.array(probs)
    print("probs shape", probs.shape)
    # use softmax to calculate cross entropy below:
    ones = np.ones([1, len(y[0])])
    m = len(y[0])
    loss = -(1.0/m) * np.sum((np.dot(y, np.log(probs)) + np.dot(ones-y, np.log(1-probs))))
    
#     log_likelihood = -np.log(probs[range(m),y])
#     loss = np.sum(log_likelihood) / m
    fit = 1/loss
    return fit
    
                

In [2]:
#load data below

with open('seeds_dataset.txt', 'r') as f:
    x = f.readlines()
data=[]

for line in x:
    temp=[]
    temp.append(line.strip().split())
    l = []
    
    for ele in temp[0]:
        l.append(float(ele))
    data.append(l)
#     print(l)
data = np.array(data)
np.random.shuffle(data)
print(data)

[[17.26   15.73    0.8763 ...  4.539   5.791   2.    ]
 [18.43   15.97    0.9077 ...  2.984   5.905   2.    ]
 [11.65   13.07    0.8575 ...  5.209   5.135   3.    ]
 ...
 [15.38   14.77    0.8857 ...  1.999   5.222   1.    ]
 [12.05   13.41    0.8416 ...  4.988   5.046   3.    ]
 [11.81   13.45    0.8198 ...  4.898   5.352   3.    ]]


In [3]:
train_X = []
train_y = []
test_X=[]
test_y=[]
len_data = len(data)
train_data = data[:int(len_data*0.8)]
test_data = data[int(len_data*0.8):]
train_X = np.array(train_data[:, [column for column in range(len(train_data[0])-1)]])
test_X  = np.array(test_data[:, [column for column in range(len(train_data[0])-1)]])
train_y = np.array([train_data[:,len(train_data[0])-1 ]])
test_y = np.array([test_data[:,len(train_data[0])-1 ]])

print(train_X.shape)
print(test_y.shape)

(168, 7)
(1, 42)


In [4]:
print(train_X[:10])

[[17.26   15.73    0.8763  5.978   3.594   4.539   5.791 ]
 [18.43   15.97    0.9077  5.98    3.771   2.984   5.905 ]
 [11.65   13.07    0.8575  5.108   2.85    5.209   5.135 ]
 [11.56   13.31    0.8198  5.363   2.683   4.062   5.182 ]
 [12.8    13.47    0.886   5.16    3.126   4.873   4.914 ]
 [18.65   16.41    0.8698  6.285   3.594   4.391   6.102 ]
 [17.36   15.76    0.8785  6.145   3.574   3.526   5.971 ]
 [19.51   16.71    0.878   6.366   3.801   2.962   6.185 ]
 [11.35   13.12    0.8291  5.176   2.668   4.337   5.132 ]
 [12.72   13.57    0.8686  5.226   3.049   4.102   4.914 ]]


In [5]:
n_classes = 3
n_examples = len(train_X)
dim_input = train_X.shape
structure = [dim_input,5,10,n_classes]

In [6]:
fit_data = GeneticAlgorithmTrainner(train_X,train_y,structure)

layers 3
_num_input: 168
random initialize weights
population id 0
layers 3
_num_input: 168
random initialize weights
population id 1
layers 3
_num_input: 168
random initialize weights
population id 2
layers 3
_num_input: 168
random initialize weights
population id 3
layers 3
_num_input: 168
random initialize weights
population id 4
layers 3
_num_input: 168
random initialize weights
population id 5
layers 3
_num_input: 168
random initialize weights
population id 6
layers 3
_num_input: 168
random initialize weights
population id 7
layers 3
_num_input: 168
random initialize weights
population id 8
layers 3
_num_input: 168
random initialize weights
population id 9
layers 3
_num_input: 168
random initialize weights
population id 10
layers 3
_num_input: 168
random initialize weights
population id 11
layers 3
_num_input: 168
random initialize weights
population id 12
layers 3
_num_input: 168
random initialize weights
population id 13
layers 3
_num_input: 168
random initialize weights
populat

x shape (168, 7) y shape (1, 168)
w shape (7, 5) bshape (168, 5)
x shape (168, 7) y shape (1, 168)
w shape (5, 10) bshape (168, 10)
x shape (168, 7) y shape (1, 168)
w shape (10, 3) bshape (168, 3)
probs shape (168, 3)
x shape (168, 7) y shape (1, 168)
w shape (7, 5) bshape (168, 5)
x shape (168, 7) y shape (1, 168)
w shape (5, 10) bshape (168, 10)
x shape (168, 7) y shape (1, 168)
w shape (10, 3) bshape (168, 3)
probs shape (168, 3)
x shape (168, 7) y shape (1, 168)
w shape (7, 5) bshape (168, 5)
x shape (168, 7) y shape (1, 168)
w shape (5, 10) bshape (168, 10)
x shape (168, 7) y shape (1, 168)
w shape (10, 3) bshape (168, 3)
probs shape (168, 3)
x shape (168, 7) y shape (1, 168)
w shape (7, 5) bshape (168, 5)
x shape (168, 7) y shape (1, 168)
w shape (5, 10) bshape (168, 10)
x shape (168, 7) y shape (1, 168)
w shape (10, 3) bshape (168, 3)
probs shape (168, 3)
x shape (168, 7) y shape (1, 168)
w shape (7, 5) bshape (168, 5)
x shape (168, 7) y shape (1, 168)
w shape (5, 10) bshape (1

NameError: name 'mean' is not defined

In [ ]:
print(fit_data)